In [3]:
print('hello wirld')

hello wirld


In [4]:
from restorant_dataset import RestDataset, lines_generator
from torchtext import data

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torch.utils.tensorboard import SummaryWriter


torch.manual_seed(1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
def get_dataset(max_examples):
    g = lines_generator()

    id_f = data.Field(sequential=False, use_vocab=False)
    stars_f = data.Field(sequential=False, use_vocab=False)
    review_f = data.Field(sequential=True, use_vocab=True)

    dataset = RestDataset(g, id_f, stars_f, review_f, max_examples)

    review_f.build_vocab(dataset)
    
    return dataset, review_f.vocab

nsamples = 50000
dataset, vocab = get_dataset(50000)

43918it [00:00, 10.46it/s]


In [6]:
batch_size = 64
iterations_per_epoch = nsamples // batch_size
print(iterations_per_epoch)
print(len(vocab))
train_iter = data.BucketIterator(
        dataset=dataset, batch_size=batch_size,
        sort_key = lambda x: len(x.review), sort=True, repeat=True)

781
7590


In [148]:
PAD = len(vocab)


def shift_left(tensor, padding_value):
    """
    tensor is 2-d, (sequence, batch)
    we are shifting the sequence and we will get (sequence+1, batch)
    """
    assert len(tensor.size()) == 2
    new_tensor = torch.full(tensor.size(), padding_value, dtype=torch.int64)
    new_tensor[0:-1, :] = tensor[1:, :]
    return new_tensor


class LSTM_LORD(nn.Module):

    def __init__(self, dim, layers, vocab_size, number_of_samples):
        super(LSTM_LORD, self).__init__()
        self.dim = dim

        self.word_embeddings = nn.Embedding(vocab_size, dim)
        
        self.stars_embedding = nn.ModuleList([nn.Embedding(2, dim) for i in range(2 * layers)]) # 2 - one for c, one for h
        self.sample_embedding = nn.ModuleList([nn.Embedding(number_of_samples, dim) for i in range(2 * layers)])
        
        # the LSTM itself
        self.lstm = nn.LSTM(dim, dim, num_layers=layers)

        # The linear layer that maps from hidden state space to word space
        self.fc = nn.Linear(dim, vocab_size)

    def forward(self, sentences, state=None):
        # sentences are shifted        
        
        w_embeds = self.word_embeddings(sentences)
        
        assert state
#             state = self.create_initial_hiddens(stars, sample_ids)
        
#         embeds = w_embeds + s_embeds + id_embeds
        
        
        lstm_out, lstm_state = self.lstm(w_embeds, state)
        logits = self.fc(lstm_out)
        probabilities = F.log_softmax(logits, dim=2)
        return probabilities, lstm_state
    
    def create_initial_hiddens(self, stars, sample_ids):
            s_embeds = [elayer(stars).unsqueeze_(0) for elayer in self.stars_embedding]
            id_embeds = [elayer(sample_ids).unsqueeze_(0) for elayer in self.sample_embedding]

            joint = [s_embed + id_embed for s_embed, id_embed in zip(s_embeds, id_embeds)]

            h = torch.cat(joint[0 : len(joint) // 2], 0)
            c = torch.cat(joint[len(joint) // 2 : ], 0)
            state = (h, c)
            return state
    



def checkpoint(model, path='/cs/labs/dshahaf/omribloch/train/text_lord/checkpoint.txt'):
#     Save:
    if os.path.exists(path):
        os.remove(path)
    torch.save(model.state_dict(), path)
    print('saved checkpoint!')

def load_checkpoint(path='/cs/labs/dshahaf/omribloch/train/text_lord/checkpoint.txt'):
#     Load:
    model = TheModelClass(*args, **kwargs)
    model.load_state_dict(torch.load(PATH))
    model.eval()
    

    
    
    




In [149]:
# model = LSTM_LORD(dim=256, layers=2, vocab_size=len(vocab)+1, number_of_samples=nsamples)
# model=model.cuda()
# loss_function = nn.NLLLoss()
# optimizer = optim.Adagrad(model.parameters())


# # training loop
# from tqdm import tqdm_notebook
# import numpy as np
# losses = []
# writer = SummaryWriter()
# for i, batch in tqdm_notebook(enumerate(train_iter)):
    
#     ids = batch.id.cuda()
#     stars = batch.stars.cuda()
#     reviews = batch.review.cuda()
    
#     model.zero_grad()
    
#     predictions, _ = model(reviews, stars, ids)
    
#     prediction_flat = predictions.view(-1, len(vocab)+1) # on cuda
#     targets_flat = shift_left(reviews, PAD).view(-1).cuda()
    
#     loss = loss_function(prediction_flat, targets_flat)
#     losses.append(loss.item())
#     writer.add_scalar('Loss/train', loss.item(), i)
    
#     prediction_flat_np = prediction_flat.detach().cpu().numpy()
#     targets_flat_np = targets_flat.detach().cpu().numpy()
# #     print(prediction_flat_np.shape)
# #     print(targets_flat_np.shape)
# #     print(targets_flat_np.size)
# #     exit(0)
#     acc = np.sum(np.argmax(prediction_flat_np, axis=1) == targets_flat_np) / targets_flat_np.size
#     writer.add_scalar('Acc/train', acc, i)

    
#     if i % iterations_per_epoch == 0:
#         print('epoch = {}'.format(i // iterations_per_epoch))
#         print(np.average(losses))
#         checkpoint(model)
#         writer.add_scalar('Loss/train_epoch', np.average(losses), i // iterations_per_epoch)
        
#     loss.backward()
#     optimizer.step()

In [150]:
def load_checkpoint(path='/cs/labs/dshahaf/omribloch/train/text_lord/checkpoint.txt'):
#     Load:
    model = LSTM_LORD(dim=256, layers=2, vocab_size=len(vocab)+1, number_of_samples=nsamples)
    model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    model.eval()
    return model

model = load_checkpoint()

In [199]:
def greedy_decode(model, vocab, stars, sample_id):
    stars = torch.tensor(stars).unsqueeze_(0)
    sample_id = torch.tensor(sample_id).unsqueeze_(0)
    token = 'START'
    state = model.create_initial_hiddens(stars, sample_id)
    print(state[0].size())
    i = 0
    tmp = [token]
    while token != 'END':
        review = torch.full((1,1), vocab.stoi[token], dtype=torch.int64)
        
        prediction, state = model(review, state)
        token = decode_int(torch.argmax(prediction))
        tmp.append(token)
        i += 1
        if i > 200:
            print('too long!!!')
            return tmp
    return tmp

with torch.no_grad():
    decoded = greedy_decode(model, vocab, 1, 500)
    sentence_final = ' '.join(decoded)
    print(sentence_final)

torch.Size([2, 1, 256])
START when i arrived brought them first . to write home truly having having having having pleasant manager him what tell tell tell anyone others anyone tell . '' '' tasted horrible service despite n't anything anything say happened happened . '' was very . ) to say neither my house tasted better places somewhere else spend anything home twice . ) ) END


In [58]:
def beam_decode(model, vocab, stars, sample_id):
    stars = torch.tensor(stars).unsqueeze_(0)
    sample_id = torch.tensor(sample_id).unsqueeze_(0)
    token = 'START'
    state = None
    i = 0
    tmp = [token]
    
    def beam_decode_inner(state, k, max_lengh):
        """
        create k new states and push into the queue. 
        """
        

tensor(2)


In [60]:
def decode_int(i):
    if i < 7590:
        return vocab.itos[i]
    else:
        return '<PAD>'

In [30]:
for i, batch in enumerate(train_iter):
    reviews = batch.review
    print(reviews.size())
    break

torch.Size([3, 64])


In [200]:
SOS_token = vocab.stoi['START']
EOS_token = vocab.stoi['END']
MAX_LENGTH = 50

stars = torch.tensor(0).unsqueeze_(0)
sample_id = torch.tensor(500).unsqueeze_(0)
token = 'START'
state = model.create_initial_hiddens(stars, sample_id)
decoded = beam_decode(decoder=model, target_tensor=torch.tensor(range(100)), decoder_hiddens=state)
print(decoded)

[[[tensor([[3]]), tensor([[163]]), tensor([[465]]), tensor([[20]]), tensor([[65]]), tensor([[28]]), tensor([[31]]), tensor([[31]]), tensor([[291]]), tensor([[22]]), tensor([[119]]), tensor([[37]]), tensor([[32]]), tensor([[31]]), tensor([[282]]), tensor([[740]]), tensor([[117]]), tensor([[142]]), tensor([[91]]), tensor([[129]]), tensor([[9]]), tensor([[1053]]), tensor([[9]]), tensor([[5]]), tensor([[254]]), tensor([[8]]), tensor([[17]]), tensor([[44]]), tensor([[657]]), tensor([[230]]), tensor([[78]]), tensor([[657]]), tensor([[230]]), tensor([[66]]), tensor([[49]]), tensor([[17]]), tensor([[137]]), tensor([[189]]), tensor([[230]]), tensor([[230]]), tensor([[230]]), tensor([[230]]), tensor([[8]]), tensor([[469]]), tensor([[28]]), tensor([[78]]), tensor([[189]]), tensor([[47]]), tensor([[168]]), tensor([[100]]), tensor([[28]]), tensor([[105]]), tensor([[131]]), tensor([[123]]), tensor([[160]]), tensor([[73]]), tensor([[71]]), tensor([[888]]), tensor([[219]]), tensor([[888]]), tensor([[2

In [201]:
l = [t.item() for t in decoded[0][0]]
print(l)
s = [vocab.itos[i] for i in l]
s_final = ' '.join(s)
print(s_final)

[3, 163, 465, 20, 65, 28, 31, 31, 291, 22, 119, 37, 32, 31, 282, 740, 117, 142, 91, 129, 9, 1053, 9, 5, 254, 8, 17, 44, 657, 230, 78, 657, 230, 66, 49, 17, 137, 189, 230, 230, 230, 230, 8, 469, 28, 78, 189, 47, 168, 100, 28, 105, 131, 123, 160, 73, 71, 888, 219, 888, 219, 10, 110, 14, 29, 15, 114, 64, 6, 63, 110, 359, 114, 466, 6, 47, 168, 168, 9, 168]
START - none of our food were were empty in us there we were being ignored by how rude staff and spoke and the menu was n't very helpful either or helpful either she did n't know anything either either either either was fine food or anything would recommend their food after any other business ever go anywhere else anywhere else to give this place _num_ stars if i could give negative stars rating i would recommend recommend and recommend


In [189]:
import operator
import torch
import torch.nn as nn
import torch.nn.functional as F
from queue import PriorityQueue
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward


def beam_decode(decoder, target_tensor, decoder_hiddens, encoder_outputs=None):
    '''
    :param target_tensor: target indexes tensor of shape [B, T] where B is the batch size and T is the maximum length of the output sentence
    :param decoder_hidden: input tensor of shape [1, B, H] for start of the decoding
    :param encoder_outputs: if you are using attention mechanism you can pass encoder outputs, [T, B, H] where T is the maximum length of input sentence
    :return: decoded_batch
    '''

    beam_width = 10
    topk = 1  # how many sentence do you want to generate
    decoded_batch = []

    # decoding goes sentence by sentence
    for idx in range(target_tensor.size(0)):
#         if isinstance(decoder_hiddens, tuple):  # LSTM case
#             decoder_hidden = (decoder_hiddens[0][:,idx, :].unsqueeze(0),decoder_hiddens[1][:,idx, :].unsqueeze(0))
#         else:
#             decoder_hidden = decoder_hiddens[:, idx, :].unsqueeze(0)
        decoder_hidden = decoder_hiddens
        
        if encoder_outputs:
            encoder_output = encoder_outputs[:,idx, :].unsqueeze(1)
        else:
            encoder_output = None

        # Start with the start of the sentence token
        decoder_input = torch.LongTensor([[SOS_token]], device=device)

        # Number of sentence to generate
        endnodes = []
        number_required = min((topk + 1), topk - len(endnodes))

        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1

        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: break

            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue

            # decode for one step using decoder
#             print(decoder_hidden[0].size())
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)#, encoder_output)
            decoder_output.squeeze_(0)

            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_width)
            nextnodes = []

            for new_k in range(beam_width):
                decoded_t = indexes[0][new_k].view(1, -1)
                #print('log_prob size is {}'.format(log_prob.size()))
                log_p = log_prob[0][new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                # increase qsize
            qsize += len(nextnodes) - 1

        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(topk)]

        utterances = []
        for score, n in sorted(endnodes, key=operator.itemgetter(0)):
            utterance = []
            utterance.append(n.wordid)
            # back trace
            while n.prevNode != None:
                n = n.prevNode
                utterance.append(n.wordid)

            utterance = utterance[::-1]
            utterances.append(utterance)

        decoded_batch.append(utterances)

    return decoded_batch
